# Acid Rain

## Analysis

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import math
import os
import glob
import datetime
import re

In [ ]:
#Import cleaned data 

nb_acid_rain = pd.read_csv("data/nb_acid_rain.csv", low_memory=False)

## Models

#### Time Series Forecast

In [ ]:
nb_acid_rain_models_1 = nb_acid_rain[(nb_acid_rain['YEAR_NO'] == 2021)].copy()

In [ ]:
nb_acid_rain_models_1["TO_DATE"] = pd.to_datetime(nb_acid_rain_models_1["TO_DATE"])

#nb_acid_rain_models_1['year'] = pd.DatetimeIndex(nb_acid_rain_models_1['TO_DATE']).year
#nb_acid_rain_models_1['month'] = pd.DatetimeIndex(nb_acid_rain_models_1['TO_DATE']).month
#nb_acid_rain_models_1['day'] = pd.DatetimeIndex(nb_acid_rain_models_1['TO_DATE']).day

nb_acid_rain_models_1.set_index('TO_DATE')

In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor

In [ ]:
nb_acid_rain_models_1 = nb_acid_rain_models_1[["STATION_NAME", "WEEK_NO","PH"]].copy()

In [ ]:
#nb_acid_rain_models_melt = nb_acid_rain_models_1.melt(id_vars="STATION_NAME", var_name="WEEK_NO", value_name="PH (pH)")

In [ ]:
nb_acid_rain_models_1 = nb_acid_rain_models_1.sort_values(['WEEK_NO',"STATION_NAME"])

#### Split the data

In [ ]:
split_point = 40
nb_acid_train = nb_acid_rain_models_1[nb_acid_rain_models_1['WEEK_NO'] < split_point].copy()
nb_acid_valid = nb_acid_rain_models_1[nb_acid_rain_models_1['WEEK_NO'] >= split_point].copy()

#### Next week's value as target/y

In [ ]:
nb_acid_train['ph_next_week'] = nb_acid_train.groupby("STATION_NAME")['PH'].shift(-1)

nb_acid_valid['ph_next_week'] = nb_acid_valid.groupby("STATION_NAME")['PH'].shift(-1)

In [ ]:
nb_acid_train['ph_next_week'] = nb_acid_train['ph_next_week'].fillna(nb_acid_train.groupby("STATION_NAME")['ph_next_week'].transform('mean'))
nb_acid_valid['ph_next_week'] = nb_acid_valid['ph_next_week'].fillna(nb_acid_valid.groupby("STATION_NAME")['ph_next_week'].transform('mean'))

##### Previous week pH value in a column

In [ ]:
nb_acid_train["lag_ph_1"] = nb_acid_train.groupby("STATION_NAME")['PH'].shift(1)
nb_acid_valid["lag_ph_1"] = nb_acid_valid.groupby("STATION_NAME")['PH'].shift(1)

In [ ]:
nb_acid_train['lag_ph_1'] = nb_acid_train['lag_ph_1'].fillna(nb_acid_train.groupby("STATION_NAME")['lag_ph_1'].transform('mean'))
nb_acid_valid['lag_ph_1'] = nb_acid_valid['lag_ph_1'].fillna(nb_acid_valid.groupby("STATION_NAME")['lag_ph_1'].transform('mean'))

##### Difference in Previous week's and this week's pH value in a column

In [ ]:
nb_acid_train["diff_ph_1"] = nb_acid_train.groupby("STATION_NAME")['PH'].diff(1)
nb_acid_valid["diff_ph_1"] = nb_acid_valid.groupby("STATION_NAME")['PH'].diff(1)

In [ ]:
nb_acid_train['diff_ph_1'] = nb_acid_train['diff_ph_1'].fillna(nb_acid_train.groupby("STATION_NAME")['diff_ph_1'].transform('mean'))
nb_acid_valid['diff_ph_1'] = nb_acid_valid['diff_ph_1'].fillna(nb_acid_valid.groupby("STATION_NAME")['diff_ph_1'].transform('mean'))

##### Mean of pH in a column

In [ ]:
nb_acid_train["mean_ph"] = nb_acid_train.groupby("STATION_NAME")['PH'].transform('mean')
nb_acid_valid["mean_ph"] = nb_acid_valid.groupby("STATION_NAME")['PH'].transform('mean')

#### Establish baseline

In [ ]:
y_pred = nb_acid_train['PH']
y_true = nb_acid_train['ph_next_week']

In [ ]:
#Export Combined Dataset to a CSV
#nb_acid_model_test = pd.concat([nb_acid_train,nb_acid_valid])
#nb_acid_model_test.to_csv("nb_acid_model_test.csv", sep=',',index=False,encoding='utf-8-sig')

In [ ]:
nb_acid_valid.isna().sum()

#### Evaluation metric

In [ ]:
def mape(y_true, y_pred):
    ape = np.abs((y_true - y_pred) / y_true)
    #ape[~np.isfinite(ape)] = 0. # VERY questionable
    ape[~np.isfinite(ape)] = 1. # pessimist estimate
    return np.mean(ape)

def wmape(y_true, y_pred):
    return np.sum(np.abs(y_true - y_pred)) / np.sum(np.abs(y_true))

In [ ]:
mape(y_true, y_pred)

In [ ]:
wmape(y_true, y_pred)

In [ ]:
features = ['PH', 'lag_ph_1', 'diff_ph_1', 'mean_ph']

#### Train the model

In [ ]:
imputer = SimpleImputer()
Xtr = imputer.fit_transform(nb_acid_train[features])
ytr = nb_acid_train['ph_next_week']


mdl = RandomForestRegressor(n_estimators=100, random_state=0, n_jobs=6)
mdl.fit(Xtr, ytr)

#### Evaluate the model

In [ ]:
Xval = imputer.transform(nb_acid_valid[features])
yval = nb_acid_valid['ph_next_week']

p = mdl.predict(Xval)

In [ ]:
mape(yval, p)

In [ ]:
wmape(yval, p)

#### Extend the model to predict n-steps

In [ ]:
nb_acid_train['ph_next_next_week'] = nb_acid_train.groupby("STATION_NAME")['PH'].shift(-2)
nb_acid_valid['ph_next_next_week'] = nb_acid_valid.groupby("STATION_NAME")['PH'].shift(-2)

In [ ]:
nb_acid_train['ph_next_next_week'] = nb_acid_train['ph_next_next_week'].fillna(nb_acid_train.groupby("STATION_NAME")['ph_next_next_week'].transform('mean'))
nb_acid_valid['ph_next_next_week'] = nb_acid_valid['ph_next_next_week'].fillna(nb_acid_valid.groupby("STATION_NAME")['ph_next_next_week'].transform('mean'))

In [ ]:
nb_acid_train = nb_acid_train.dropna(subset=['ph_next_week','ph_next_next_week'])

In [ ]:
imputer = SimpleImputer()
Xtr = imputer.fit_transform(nb_acid_train[features])
ytr = nb_acid_train[['ph_next_week', 'ph_next_next_week']]

mdl = RandomForestRegressor(n_estimators=100, random_state=0, n_jobs=6)
mdl.fit(Xtr, ytr)

In [ ]:
Xval = imputer.transform(nb_acid_valid[features])
yval = nb_acid_valid[['ph_next_week', 'ph_next_next_week']]

p = mdl.predict(Xval)

In [ ]:
mdl.score(Xval, yval)

#### Predicting new examples

In [ ]:
new_examples = nb_acid_valid[nb_acid_valid['WEEK_NO'] == 48].copy()

In [ ]:
new_examples = new_examples.dropna()

In [ ]:
p = mdl.predict(new_examples[features])

In [ ]:
new_examples['p_ph_next_week'] = p[:, 0]
new_examples['p_ph_next_next_week'] = p[:, 1]

In [ ]:
new_examples.head()